### Один вариант реализации

In [5]:
def singleton(cls):    
    instance = [None]
    def wrapper(*args, **kwargs):
        if instance[0] is None:
            instance[0] = cls(*args, **kwargs)
        return instance[0]

    return wrapper

In [6]:
@singleton
class SomeSingletonClass:
    x = 2
    def __init__(self):
        print("Created!")

instance = SomeSingletonClass()  # prints: Created!
instance = SomeSingletonClass()  # doesn't print anything
print(instance.x)                # 2

instance.x = 3
print(SomeSingletonClass().x)    # 3

Created!
2
3


### Другой вариант реализации

In [6]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance
s = Singleton()
print("Object created", s)
s1 = Singleton()
print("Object created", s1)

print(s == s1)

Object created <__main__.Singleton object at 0x0000014FCC16E7C0>
Object created <__main__.Singleton object at 0x0000014FCC16E7C0>
True


### Пример применения

In [7]:
from abc import ABC, abstractmethod
class SingletonMeta(type):

    """Метакласс для класса Parser"""

    _instances = {}

 
    def __call__(cls, *args, **kwargs):

        """

        При инициализации класса Parser в словарь _instances заносится ключ с названием файла, который парсится, и

        значение с объектом парсера. При попытке иницилизировать новый объект Parser для уже рапрасенного файла вместо

        создания нового объекта вернется уже существующий объект

        :return: объект класса-наследника

        """

        file_path = Path(kwargs['file']['path'])

        file_name = file_path.name

 
        if file_name not in cls._instances:

            instance = super().__call__(*args, **kwargs)

            cls._instances[file_name] = instance

 
        return cls._instances[file_name]

class ParserInterface(ABC):

    __metaclass__ = SingletonMeta
 

    @abstractmethod

    def calculate(self, *args, **kwargs):

        """Метод для расчета расхода газа в случае свода VLP для газлифта и мощности на СУ для скважины, оборудованной

        УЭЦН"""

 
    @abstractmethod

    def well_properties(self, *args, **kwargs) -> dict:

        """Метод для получения данных до VLP таблицы (PVT,Режим и тд)"""

        pass


    @abstractmethod

    def get_vlp(self, *args, **kwargs):

        """Метод, возвращающий vlp таблицу"""

In [13]:
import numpy as np
import pandas as pd

class TRParser(ParserInterface):

    def __init__(self, file: dict, vlp_parser_obj):

        """
        :param file: словарь {
            'path': путь к файлу,
            'sheet': имя листа
        }

        :param vlp_parser_obj: объект одного из классов Parser или ParserESP
        """
        self.vlp_parser = vlp_parser_obj
        self.__raw_sheet = pd.read_excel(io=file['path'], sheet_name=file['sheet'], engine='openpyxl')
        self.__raw_sheet.dropna(axis=1, how='all', inplace=True)
        self.__raw_sheet.dropna(how='all', inplace=True)
         # индекс заголовков
        header_i = 0
        while pd.isnull(self.__raw_sheet.iloc[header_i, 0]):

            header_i += 1
        self.__raw_sheet = pd.read_excel(io=file['path'], sheet_name=file['sheet'], header=header_i + 2,
                                         engine='openpyxl')
        self.__raw_sheet.dropna(axis=1, how='all', inplace=True)
        self.__raw_sheet.dropna(how='all', inplace=True)
        self.__raw_sheet.columns = [col.replace('\n', ' ').replace('-', '') for col in self.__raw_sheet.columns]
        self.wells_list = self.__raw_sheet['№ скв'].dropna().values
        self.__cols = self.__raw_sheet.columns.tolist()
        for i in range(1, len(self.__cols) - 1):
            if 'Unnamed:' in self.__cols[i]:
                self.__cols[i] = self.__cols[i - 1]

        # формирование заголовка таблицы
        raw_headers = self.__raw_sheet.iloc[0: 3, :]
        self.__sub_header = raw_headers.iloc[0, :]
        self.__sub_header = self.__sub_header.fillna(raw_headers.iloc[1, :])
        self.__sub_header = pd.Series(
            data=[col.replace('\n', ' ').replace('- ', '').replace('-', '') if isinstance(col, str)
                  else col for col in self.__sub_header.values], index=self.__raw_sheet.columns
        )

 
    def calculate(self, pwf: Union[np.ndarray, list, tuple], rates: Union[np.ndarray, list, tuple],
                  name: str, plin: float = None) -> np.ndarray:
        """
        Метод рассчитывает энергозатраты(расход газа, частота вращения) на обеспечение режима (забойное давление,
        дебит)
        :param plin: значение линейного давления, атм
        :param name: имя скважины / id насоса (в зависимости от подаваемой таблицы)
        :param pwf: массив значений забойного давления, соотвествующих значениям дебита, атм
        :param rates: массив значений забойного давления, соотвествующих значениям дебита, атм
        :return: массив значений расхода газа/частоты вращения
        """
        return self.vlp_parser.calculate(pwf, rates, name, plin)

    def get_vlp(self, well_name: str, pressure: float = None) -> pd.DataFrame:

        """
        Возвращает таблицу с значениями искомого параметра
        :param pressure: значение линейного давления для таблиц ТР
        :param well_name: название искомого объекта (id насоса, номер скважины)
        :return: таблица с значениями искомого параметра
        """
        return self.vlp_parser.get_vlp(well_name, pressure)
 
    def get_well_index(self, well_num: str) -> int:
        """
         Метод возвращает порядковый номер строки искомой скважины в датафрейме техрежима
        :param well_num: номер скважины (столбец "№ скв")
        :return: порядковый номер строки искомой скважины
        """
        idx = self.__raw_sheet['№ скв'][self.__raw_sheet['№ скв'] == well_num].index
        if idx.any():
            return idx
        raise ValueError(f'Скважина "{well_num}" не найдена')